In [14]:
import pandas as pd
import json
from colorama import Style

parquet_file_path = 'data/GA_ITS_MSA_20221021_JUV_EMP_HIS2.parquet'

df = pd.read_parquet(parquet_file_path)

output_json = json.load(open('generate/json_output_dump 2.json'))

In [15]:
df.pred.unique()

array(['General information about game / location',
       'Comment about the previous action',
       'Description of action (e.g. Player A to Player B / Player A shoots etc.)',
       'Statistics of game', 'Statistics of player',
       'Gossip [Player / Game]', 'Substitution', 'VAR',
       'Statistics of team'], dtype=object)

In [16]:
df.head(50)

,start,end,text,previous_text_2,previous_text,next_text,next_text_2,pred
500,2333110,2335020,We approach the half hour mark.,,,The goal came early.,And it came courtesy of Moyes Keane.,General information about game / location
501,2335120,2335980,The goal came early.,,We approach the half hour mark.,And it came courtesy of Moyes Keane.,Inside eight minutes.,Comment about the previous action
502,2336420,2338020,And it came courtesy of Moyes Keane.,We approach the half hour mark.,The goal came early.,Inside eight minutes.,"A good run, but all about the quality of the c...",Description of action (e.g. Player A to Player...
503,2340200,2341260,Inside eight minutes.,The goal came early.,And it came courtesy of Moyes Keane.,"A good run, but all about the quality of the c...",Just continues to impress.,Statistics of game
504,2342260,2346300,"A good run, but all about the quality of the c...",And it came courtesy of Moyes Keane.,Inside eight minutes.,Just continues to impress.,"Game number 15 for him and Danilo, the only tw...",Description of action (e.g. Player A to Player...
505,2348160,2349920,Just continues to impress.,Inside eight minutes.,"A good run, but all about the quality of the c...","Game number 15 for him and Danilo, the only tw...",Bonucci.,Comment about the previous action
506,2354320,2363560,"Game number 15 for him and Danilo, the only tw...","A good run, but all about the quality of the c...",Just continues to impress.,Bonucci.,Seemingly responding there to some of the whis...,Statistics of player
507,2366660,2367060,Bonucci.,Just continues to impress.,"Game number 15 for him and Danilo, the only tw...",Seemingly responding there to some of the whis...,Allegri was diplomatic in his communication ab...,Statistics of player
508,2371420,2374760,Seemingly responding there to some of the whis...,"Game number 15 for him and Danilo, the only tw...",Bonucci.,Allegri was diplomatic in his communication ab...,Said he's 35.,Comment about the previous action
509,2384320,2388380,Allegri was diplomatic in his communication ab...,Bonucci.,Seemingly responding there to some of the whis...,Said he's 35.,We do have to manage him.,Comment about the previous action


In [17]:
df[(df.pred == 'Statistics of team') & (df.index == 967)]

,start,end,text,previous_text_2,previous_text,next_text,next_text_2,pred
967,4997300,5002060,Some of Juventus' supporting cast now chipping...,"Not the same prestige as Paris Saint-Germain, ...",And his first in Serie A since January.,Keane for McKinney.,Great stretch to control.,Statistics of team


In [18]:
print(df.loc[967].text)

Some of Juventus' supporting cast now chipping in, lightening the load on Vlahovic.


In [19]:
(df.loc[967].end - df.loc[967].start) / 1000

4.76

## Cosine similarity

In [20]:
from sklearn.metrics.pairwise import cosine_similarity
from nltk.tokenize import word_tokenize
import string
import warnings
import gensim
import gensim.downloader as api
warnings.filterwarnings("ignore")

import fasttext

fasttext_model300 = api.load('fasttext-wiki-news-subwords-300')

def preprocess_and_calculate_similarity(sentence1, sentence2, model):
    def preprocess_text(text):
        text = text.lower()
        
        text = text.translate(str.maketrans("", "", string.punctuation))
        
        tokens = word_tokenize(text)
        
        return tokens

    tokens1 = preprocess_text(sentence1)
    tokens2 = preprocess_text(sentence2)

    def sentence_vector(sentence, model):
        vector = [model[word] for word in sentence if word in model]
        return sum(vector) / len(vector) if vector else None
    
    vector1 = sentence_vector(tokens1, model)
    vector2 = sentence_vector(tokens2, model)

    if vector1 is not None and vector2 is not None:
        vector1 = vector1.reshape(1, -1)
        vector2 = vector2.reshape(1, -1)

        similarity_score = cosine_similarity(vector1, vector2)[0, 0]

        return similarity_score
    else:
        return None

In [74]:
import gensim.downloader as api
from sklearn.metrics.pairwise import cosine_similarity
from nltk.tokenize import word_tokenize
import string
import warnings
warnings.filterwarnings("ignore")

word2vec_model = api.load('word2vec-google-news-300')

def preprocess_and_calculate_similarity(sentence1, sentence2, word2vec_model):
    def preprocess_text(text):
        text = text.lower()
        
        text = text.translate(str.maketrans("", "", string.punctuation))
        
        tokens = word_tokenize(text)
        
        return tokens

    tokens1 = preprocess_text(sentence1)
    tokens2 = preprocess_text(sentence2)

    def sentence_vector(sentence, model):
        vector = [model[word] for word in sentence if word in model]
        return sum(vector) / len(vector) if vector else None

    vector1 = sentence_vector(tokens1, word2vec_model)
    vector2 = sentence_vector(tokens2, word2vec_model)

    if vector1 is not None and vector2 is not None:
        vector1 = vector1.reshape(1, -1)
        vector2 = vector2.reshape(1, -1)

        similarity_score = cosine_similarity(vector1, vector2)[0, 0]

        return similarity_score
    else:
        return None

In [21]:
highest_sim = 0
highest_sim_text = ''
previous_text = df.loc[967].previous_text_2
for i in output_json['496']['statistics']:
    try: 
        print(output_json['496']['statistics'][i]['short'])
        if preprocess_and_calculate_similarity(previous_text, output_json['496']['statistics'][i]['short'], fasttext_model300) > highest_sim:
            highest_sim = preprocess_and_calculate_similarity(previous_text, output_json['496']['statistics'][i]['short'], fasttext_model300)
            highest_sim_text = output_json['496']['statistics'][i]['short']
    except:
        print(output_json['496']['statistics'][i]['Short'])
        if preprocess_and_calculate_similarity(previous_text, output_json['496']['statistics'][i]['Short'], fasttext_model300) > highest_sim:
            highest_sim = preprocess_and_calculate_similarity(previous_text, output_json['496']['statistics'][i]['Short'], fasttext_model300)
            highest_sim_text = output_json['496']['statistics'][i]['Short']

print()
print(f'{Style.BRIGHT}score: {round(highest_sim, 2)}, text: {highest_sim_text}{Style.RESET_ALL}')

Ah, Juventus, with their consistent 'ups and downs' through 38 games, they've really outdone themselves with 22 wins.
Wow, Juventus 'shattered' records by scoring a 'whopping' 56 goals this season, an 'absolutely earth-shattering' average of just 1.5 per game.
Oh, how convenient! They score the most and also let the most goals in between minutes 46-60. Talk about balance!
Wow, Juventus with 8 glorious wins, right after they took a quick break for 2 draws and 3 losses, how refreshing!
Clearly, our team has a knack for the dramatic, gathering the highest percentage of yellow cards towards the end of the game and apparently saving red cards for bonus time.
Wow, we should all be shaking at Juventus' powerful record of scoring on all their three penalties this season.
Oh, bravo Juventus! A whopping 20 clean sheets, evenly split between home and away. Such perfectionists!

score: 0.8899999856948853, text: Oh, how convenient! They score the most and also let the most goals in between minutes 

In [15]:
print(f'{Style.BRIGHT}Example output{Style.RESET_ALL}')
print()
print(df.loc[967].previous_text_2, df.loc[967].previous_text)
print(f'{Style.BRIGHT}{highest_sim_text}{Style.RESET_ALL}')
print(df.loc[967].next_text, df.loc[967].next_text_2)

Example output

Not the same prestige as Paris Saint-Germain, but an important goal nonetheless from Wes McKinney. And his first in Serie A since January.
Oh, how convenient! They score the most and also let the most goals in between minutes 46-60. Talk about balance!
Keane for McKinney. Great stretch to control.


# Player

In [22]:
df[(df.pred == 'Statistics of player') & (df.index == 890)]

,start,end,text,previous_text_2,previous_text,next_text,next_text_2,pred
890,4677160,4681760,Game 4-2-2 in Serie A for Leo Bonucci.,Cuadrado couldn't keep it in play.,Looks like another off night for the Juventus ...,"My maths isn't great, but that's the equivalen...",Cuadrado wins the header.,Statistics of player


In [23]:
(df.loc[890].end - df.loc[890].start) / 1000

4.6

In [24]:
4677160 / 1000 / 60

77.95266666666666

## Cosine similarity

In [52]:
highest_sim = 0
highest_sim_text = ''
previous_text = df.loc[890].previous_text_2
for i in output_json['496']['players']['866']['statistics']:
    try: 
        long_string = output_json['496']['players']['866']['statistics'][i]['short']
        score = preprocess_and_calculate_similarity(previous_text, output_json['496']['players']['866']['statistics'][i]['short'], fasttext_model300)
        max_length = 80
        broken_lines = [long_string[j:j+max_length] for j in range(0, len(long_string), max_length)]
        for idx, line in enumerate(broken_lines):
            print('- ' if idx == 0 else '  ', line, f'score: {score}' if idx + 1 == len(broken_lines) else '')
        if score > highest_sim:
            highest_sim = preprocess_and_calculate_similarity(previous_text, output_json['496']['players']['866']['statistics'][i]['short'], fasttext_model300)
            highest_sim_text = output_json['496']['players']['866']['statistics'][i]['short']
    except:
        long_string = output_json['496']['players']['866']['statistics'][i]['Short']
        score = preprocess_and_calculate_similarity(previous_text, output_json['496']['players']['866']['statistics'][i]['Short'], fasttext_model300)
        max_length = 80
        broken_lines = [long_string[j:j+max_length] for j in range(0, len(long_string), max_length)]
        for idx, line in enumerate(broken_lines):
            print('- ' if idx == 0 else '  ', line, f'score: {score}' if idx + 1 == len(broken_lines) else '')
        if score > highest_sim:
            highest_sim = preprocess_and_calculate_similarity(previous_text, output_json['496']['players']['866']['statistics'][i]['Short'], fasttext_model300)
            highest_sim_text = output_json['496']['players']['866']['statistics'][i]['Short']

score = round(highest_sim, 2)
text = highest_sim_text.replace(", making", ", \nmaking")

print()
print(f'{Style.BRIGHT}Example commentary{Style.RESET_ALL}')
print('"' + df.loc[890].previous_text_2, df.loc[890].previous_text)
text = highest_sim_text.replace(", making", ", \nmaking")
print(f'{Style.BRIGHT}{text}{Style.RESET_ALL}')
print(df.loc[890].next_text, df.loc[890].next_text_2 + '"')

-  Oh wow, isn't it miraculous, Cuadrado Bello has actually shown up in 31 games th 
   is season, making it to the starting lineup in 24! So impressive! score: 0.8943054676055908
-  2087 minutes in, and Cuadrado Bello still thinks he's playing a never-ending foo 
   tball marathon. score: 0.8280103206634521
-  Oh look, Cuadrado Bello rules the midfield now! score: 0.8162208199501038
-  Oh, a 7.1 rating, Cuadrado Bello must really be the crown jewel of his team. score: 0.8031725883483887
-  Oh right, Cuadrado Bello, the non-captain captain. Quite the influencer! score: 0.7804974317550659
-  Star of the season, Bello, diligently makes his exit - a whopping 13 times early 
    and 7 substitutions, truly his team's secret weapon. score: 0.7603869438171387
-  Cuadrado Bello, hitting the target 11 out of 27 times - clearly failures don't p 
   hase him! score: 0.7972791194915771
-  Wow, surely Cuadrado Bello's whopping one goal and three assists this season hav 
   e the fans rejoicing in e

Example commentary
"Cuadrado couldn't keep it in play. Looks like another off night for the Juventus legend.
Oh wow, isn't it miraculous, Cuadrado Bello has actually shown up in 31 games this season, 
making it to the starting lineup in 24! So impressive!
My maths isn't great, but that's the equivalent of 11 full seasons, I believe. Cuadrado wins the header."


### Cosine similarity with direct text generation with flavor

In [66]:
df.head(50)

,start,end,text,previous_text_2,previous_text,next_text,next_text_2,pred
500,2333110,2335020,We approach the half hour mark.,,,The goal came early.,And it came courtesy of Moyes Keane.,General information about game / location
501,2335120,2335980,The goal came early.,,We approach the half hour mark.,And it came courtesy of Moyes Keane.,Inside eight minutes.,Comment about the previous action
502,2336420,2338020,And it came courtesy of Moyes Keane.,We approach the half hour mark.,The goal came early.,Inside eight minutes.,"A good run, but all about the quality of the c...",Description of action (e.g. Player A to Player...
503,2340200,2341260,Inside eight minutes.,The goal came early.,And it came courtesy of Moyes Keane.,"A good run, but all about the quality of the c...",Just continues to impress.,Statistics of game
504,2342260,2346300,"A good run, but all about the quality of the c...",And it came courtesy of Moyes Keane.,Inside eight minutes.,Just continues to impress.,"Game number 15 for him and Danilo, the only tw...",Description of action (e.g. Player A to Player...
505,2348160,2349920,Just continues to impress.,Inside eight minutes.,"A good run, but all about the quality of the c...","Game number 15 for him and Danilo, the only tw...",Bonucci.,Comment about the previous action
506,2354320,2363560,"Game number 15 for him and Danilo, the only tw...","A good run, but all about the quality of the c...",Just continues to impress.,Bonucci.,Seemingly responding there to some of the whis...,Statistics of player
507,2366660,2367060,Bonucci.,Just continues to impress.,"Game number 15 for him and Danilo, the only tw...",Seemingly responding there to some of the whis...,Allegri was diplomatic in his communication ab...,Statistics of player
508,2371420,2374760,Seemingly responding there to some of the whis...,"Game number 15 for him and Danilo, the only tw...",Bonucci.,Allegri was diplomatic in his communication ab...,Said he's 35.,Comment about the previous action
509,2384320,2388380,Allegri was diplomatic in his communication ab...,Bonucci.,Seemingly responding there to some of the whis...,Said he's 35.,We do have to manage him.,Comment about the previous action


In [21]:
print(f'{Style.BRIGHT}player statistic with previous text as context{Style.RESET_ALL}')
df.loc[505].previous_text_2, df.loc[505].previous_text, df.loc[505].text, df.loc[505].next_text, df.loc[505].next_text_2

player statistic with previous text as context


('Inside eight minutes.',
 'A good run, but all about the quality of the cross from Filip Kostic.',
 'Just continues to impress.',
 'Game number 15 for him and Danilo, the only two Juventus players, indeed, to feature in every single match for the old lady thus far this term.',
 'Bonucci.')

In [22]:
print(f'{Style.BRIGHT}time length of statistic comment{Style.RESET_ALL}')
print(f'{(df.loc[507].end - df.loc[506].start) / 1000}s')
print(f'length: long')

time length of statistic comment
12.74s
length: long


In [13]:
with open('generate/output_team_1 2.json') as f:
    input_json = json.load(f)

In [49]:
f'{df.loc[505].previous_text_2} {df.loc[505].previous_text}'

'Inside eight minutes. A good run, but all about the quality of the cross from Filip Kostic.'

In [53]:
highest_sim = 0
highest_sim_text = ''
previous_text = f'{df.loc[505].previous_text_2}'

print(f'{Style.BRIGHT}Previous text: {previous_text} {Style.RESET_ALL}')

for statistic in input_json['496']['players']['1821']['statistics'].keys():
    print(round(preprocess_and_calculate_similarity(previous_text, input_json['496']['players']['1821']['statistics'][statistic], fasttext_model300), 2), input_json['496']['players']['1821']['statistics'][statistic])
    if round(preprocess_and_calculate_similarity(previous_text, input_json['496']['players']['1821']['statistics'][statistic], fasttext_model300), 2) > highest_sim:
            highest_sim = round(preprocess_and_calculate_similarity(previous_text, input_json['496']['players']['1821']['statistics'][statistic], fasttext_model300), 2)
            highest_sim_text = input_json['496']['players']['1821']['statistics'][statistic]

print()
print(f'{Style.BRIGHT}score: {round(highest_sim, 2)}, text: {highest_sim_text}{Style.RESET_ALL}')

Previous text: Inside eight minutes. 
0.67 And on the field, we have Kostić, the notable Midfielder.
0.78 Kostić has been seen in 37 games this season, starting in 33 of them and playing a total of 2,582 minutes.
0.65 Despite his solid performance, Kostić is not the team captain.
0.73 He has been pulled from the bench 4 times, replaced in 21 instances and has been a bench player in 5 games.
0.74 Kostić has taken 27 shots this season, with 11 on target, leading to a total of 3 goals.
0.78 Showing his skill in ball distribution, Kostić has made 875 total passes, with 67 key passes and an accuracy of 18%.
0.74 On the defensive side, he's made a total of 41 tackles, with 4 blocks and 16 interceptions.
0.73 He's a powerful player, engaged in 197 duels and won 89. He's made 47 dribbling attempts with 23 successful attempts.
0.76 Kostić has drawn 12 fouls and committed 29, receiving 3 yellow cards in the process.
0.66 Regarding penalties, he hasn't scored, missed, won or committed any penalti

In [55]:
highest_sim = 0
highest_sim_text = ''
previous_text = f'{df.loc[505].previous_text_2} {df.loc[505].previous_text}'

print(f'{Style.BRIGHT}Previous text: {previous_text} {Style.RESET_ALL}')

for statistic in input_json['496']['players']['1821']['statistics']:
    print(round(preprocess_and_calculate_similarity(previous_text, input_json['496']['players']['1821']['statistics'][statistic], fasttext_model300), 2), input_json['496']['players']['1821']['statistics'][statistic])
    if round(preprocess_and_calculate_similarity(previous_text, input_json['496']['players']['1821']['statistics'][statistic], fasttext_model300), 2) > highest_sim:
            highest_sim = round(preprocess_and_calculate_similarity(previous_text, input_json['496']['players']['1821']['statistics'][statistic], fasttext_model300), 2)
            highest_sim_text = input_json['496']['players']['1821']['statistics'][statistic]

print()
print(f'{Style.BRIGHT}score: {round(highest_sim, 2)}, \ntext: {highest_sim_text}{Style.RESET_ALL}')

Previous text: Inside eight minutes. A good run, but all about the quality of the cross from Filip Kostic. 
0.9 And on the field, we have Kostić, the notable Midfielder.
0.91 Kostić has been seen in 37 games this season, starting in 33 of them and playing a total of 2,582 minutes.
0.87 Despite his solid performance, Kostić is not the team captain.
0.85 He has been pulled from the bench 4 times, replaced in 21 instances and has been a bench player in 5 games.
0.87 Kostić has taken 27 shots this season, with 11 on target, leading to a total of 3 goals.
0.91 Showing his skill in ball distribution, Kostić has made 875 total passes, with 67 key passes and an accuracy of 18%.
0.86 On the defensive side, he's made a total of 41 tackles, with 4 blocks and 16 interceptions.
0.81 He's a powerful player, engaged in 197 duels and won 89. He's made 47 dribbling attempts with 23 successful attempts.
0.83 Kostić has drawn 12 fouls and committed 29, receiving 3 yellow cards in the process.
0.82 Regard

In [79]:
from openai import OpenAI

def generate_gpt_flavor(input):
        client = OpenAI(
            # defaults to os.environ.get("OPENAI_API_KEY")
            api_key="API-KEY",
        )

        response = client.chat.completions.create(
            messages=[
                {"role": "system", "content": "You get a text from a sports journalist for live commentary. Take the text and create a output with a length of maximum 3 sentences. Write the text with the provided flavor."},
                {"role": "user", "content": f"{input}"},
            ],
            model="gpt-4",
            temperature=0.9,
        )
        return response.choices[0].message.content



In [59]:
flavor = 'neutral'
input_text = f'Create a text with the flavor {flavor}. This is the text that needs to be written in 3 sentences: {highest_sim_text}. To provide you context. This is the previous text: {previous_text}.'

output_text = generate_gpt_flavor(input_text)

In [78]:
print(f'{Style.BRIGHT}Example with generation:{Style.RESET_ALL}')
print()
print(f'{df.loc[505].previous_text_2} {df.loc[505].previous_text}')
new_output_text = output_text.replace('. ', '. \n')
print(f"{Style.BRIGHT}{new_output_text}{Style.RESET_ALL}")
print(df.loc[507].text, df.loc[508].text)

Example with generation:

Inside eight minutes. A good run, but all about the quality of the cross from Filip Kostic.
The dedication and discipline of Kostić this season have been nothing short of astonishing, as he has taken to the field in 37 games, starting in 33. 
This commitment to his team and sport has resulted in an impressive tally of 2,582 minutes of playtime, a testament to his stamina as well as his love for the game. 
Despite the tremendous physical toll, he shows no signs of backing down, proving once again his determination to pursue excellence in his craft. 
Even after eight minutes into the match, it's clear he's ready to meet any challenge head-on.
Bonucci. Seemingly responding there to some of the whistles from the home crowd.


In [80]:
flavor = 'neutral'
input_text = f'Create a text with the flavor {flavor}. This is the text that needs to be written in maximum 3 sentences: {highest_sim_text}. To provide you context. This is the previous text: {previous_text}.'

shorter_output_text = generate_gpt_flavor(input_text)

In [81]:
print(f'{Style.BRIGHT}Example with generation:{Style.RESET_ALL}')
print()
print(f'{df.loc[505].previous_text_2} {df.loc[505].previous_text}')
new_output_text = shorter_output_text.replace('. ', '. \n')
print(f"{Style.BRIGHT}{new_output_text}{Style.RESET_ALL}")
print(df.loc[507].text, df.loc[508].text)

Example with generation:

Inside eight minutes. A good run, but all about the quality of the cross from Filip Kostic.
Kostić has made a remarkable appearance in 37 games this season, demonstrating his unwavering commitment. 
With a starting position in 33 games, he has logged an impressive 2,582 minutes on the field. 
Despite the rigorous schedule, his performance has not waned.
Bonucci. Seemingly responding there to some of the whistles from the home crowd.


### whole player statistics with flavor.

In [69]:
highest_sim = 0
highest_sim_text = ''
previous_text = f'{df.loc[505].previous_text_2}'

print(f'{Style.BRIGHT}Previous text: {previous_text} {Style.RESET_ALL}')

for statistic in output_json['496']['players']['1821']['statistics']:
    try:
        print(round(preprocess_and_calculate_similarity(previous_text, output_json['496']['players']['1821']['statistics'][statistic]['long'], fasttext_model300), 2), output_json['496']['players']['1821']['statistics'][statistic]['long'])
        if round(preprocess_and_calculate_similarity(previous_text, output_json['496']['players']['1821']['statistics'][statistic]['long'], fasttext_model300), 2) > highest_sim:
                highest_sim = round(preprocess_and_calculate_similarity(previous_text, output_json['496']['players']['1821']['statistics'][statistic]['long'], fasttext_model300), 2)
                highest_sim_text = output_json['496']['players']['1821']['statistics'][statistic]['long']
    except:
        print(round(preprocess_and_calculate_similarity(previous_text, output_json['496']['players']['1821']['statistics'][statistic]['Long'], fasttext_model300), 2), output_json['496']['players']['1821']['statistics'][statistic]['Long'])
        if round(preprocess_and_calculate_similarity(previous_text, output_json['496']['players']['1821']['statistics'][statistic]['Long'], fasttext_model300), 2) > highest_sim:
                highest_sim = round(preprocess_and_calculate_similarity(previous_text, output_json['496']['players']['1821']['statistics'][statistic]['Long'], fasttext_model300), 2)
                highest_sim_text = output_json['496']['players']['1821']['statistics'][statistic]['Long']

print()
print(f'{Style.BRIGHT}score: {round(highest_sim, 2)}, text: {highest_sim_text}{Style.RESET_ALL}')

Previous text: Inside eight minutes. 
0.72 Oh, brace yourselves, folks. Kostić, the supposedly 'remarkable' midfielder has decided to show up on the field. Now, isn't that a pleasant surprise? I mean, when has he ever failed to astonish us with his 'impressive' performances? Everything from his 'precise' passes, to his 'exceptional' shots, never cease to amaze. Surely, we're all just holding our breaths, eagerly waiting for what kind of 'mind-blowing' strategy he's going to pull off next. Do keep us entertained, Kostić.
0.74 In a surprise to absolutely nobody, Kostić has been seen - yes, actually spotted - in 37 games this season. And, brace yourselves, he even started in 33 of them. Truly, a show of fortitude and unparalleled commitment to the cause. His perseverance has seen him clock in an awe-inspiring 2,582 minutes on the field. It's almost as if he's being paid to do this. A round of applause for Kostić and his ability to show up and do his job. We're all truly amazed.
0.7 Well, 

In [75]:
print(f'{Style.BRIGHT}Example with generation:{Style.RESET_ALL}')
print()
print(f'{df.loc[505].previous_text_2} {df.loc[505].previous_text}')
new_output_text = highest_sim_text.replace('. ', '. \n')
print(f"{Style.BRIGHT}{new_output_text}{Style.RESET_ALL}")
print(df.loc[507].text, df.loc[508].text)

Example with generation:

Inside eight minutes. A good run, but all about the quality of the cross from Filip Kostic.
Let's take a moment to acknowledge our shining star who's spent more time warming the bench than scoring points. 
It takes a special kind of talent to get picked from the bench 4 times, no easy feat. 
But let's not forget the pride of being substituted 21 whopping times! Oh, and the icing on the cake? Being a professional bench player in 5 games. 
A massive round of applause for our unsung hero, the bench never felt warmer!
Bonucci. Seemingly responding there to some of the whistles from the home crowd.


In [76]:
highest_sim = 0
highest_sim_text = ''
previous_text = f'{df.loc[505].previous_text_2}'

print(f'{Style.BRIGHT}Previous text: {previous_text} {Style.RESET_ALL}')

for statistic in output_json['496']['players']['1821']['statistics']:
    try:
        print(round(preprocess_and_calculate_similarity(previous_text, output_json['496']['players']['1821']['statistics'][statistic]['middle'], fasttext_model300), 2), output_json['496']['players']['1821']['statistics'][statistic]['middle'])
        if round(preprocess_and_calculate_similarity(previous_text, output_json['496']['players']['1821']['statistics'][statistic]['middle'], fasttext_model300), 2) > highest_sim:
                highest_sim = round(preprocess_and_calculate_similarity(previous_text, output_json['496']['players']['1821']['statistics'][statistic]['middle'], fasttext_model300), 2)
                highest_sim_text = output_json['496']['players']['1821']['statistics'][statistic]['middle']
    except:
        print(round(preprocess_and_calculate_similarity(previous_text, output_json['496']['players']['1821']['statistics'][statistic]['Middle'], fasttext_model300), 2), output_json['496']['players']['1821']['statistics'][statistic]['Middle'])
        if round(preprocess_and_calculate_similarity(previous_text, output_json['496']['players']['1821']['statistics'][statistic]['Middle'], fasttext_model300), 2) > highest_sim:
                highest_sim = round(preprocess_and_calculate_similarity(previous_text, output_json['496']['players']['1821']['statistics'][statistic]['Middle'], fasttext_model300), 2)
                highest_sim_text = output_json['496']['players']['1821']['statistics'][statistic]['Middle']

print()
print(f'{Style.BRIGHT}score: {round(highest_sim, 2)}, text: {highest_sim_text}{Style.RESET_ALL}')

Previous text: Inside eight minutes. 
0.67 Well, well, well, isn't it a surprise? Kostić, the 'stellar' midfielder gracing us with his presence on the field. I really can't wait to see what 'groundbreaking' strategy he employs next.
0.77 In an astonishing turn of events, Kostić has showed us his unyielding dedication by appearing in 37 games this season. Starting in 33 of them, he's clocked in an overwhelming 2,582 minutes of playtime. Man, he must be exhausted.
0.69 In a surprising turn of events, Kostić brought his A-game today. You'd think this might earn him the captain's armband, but alas, wishful thinking has never been a strong suit in football, has it?
0.73 Ah yes, let's praise our star player who's been yanked from the bench 4 times. 21 substitutions and running benchwarmer for 5 games? What a record to behold!
0.74 Well, isn't Kostić just an inspiration for all aspiring footballers? He's taken 27 shots this season and only 11 of them actually located the target. Bonus point? 

In [77]:
print(f'{Style.BRIGHT}Example with generation:{Style.RESET_ALL}')
print()
print(f'{df.loc[505].previous_text_2} {df.loc[505].previous_text}')
new_output_text = highest_sim_text.replace('. ', '. \n')
print(f"{Style.BRIGHT}{new_output_text}{Style.RESET_ALL}")
print(df.loc[507].text, df.loc[508].text)

Example with generation:

Inside eight minutes. A good run, but all about the quality of the cross from Filip Kostic.
In an astonishing turn of events, Kostić has showed us his unyielding dedication by appearing in 37 games this season. 
Starting in 33 of them, he's clocked in an overwhelming 2,582 minutes of playtime. 
Man, he must be exhausted.
Bonucci. Seemingly responding there to some of the whistles from the home crowd.


In [83]:
input_json

{'496': {'name': 'Juventus',
  'information': '"Juventus, represented by the code JUV, hails from the beautiful country of Italy. This sturdy team, founded in the rich sporting history of 1897, is set to showcase its prowess on the field today."',
  'statistics': {'team_performance': "Juventus, with their form 'WDDWDDLWLWWWWWWWWLDLWWWWLWWWLLLDWWWLLW', have been showing consistent performance. With a total of 38 matches played, they have 22 victories, where 13 are at home and 9 are away games.",
   'team_goals': 'The total number of goals that Juventus scored is impressive at 56, with 38 being scored in home fixtures and 18 in away games. Their average goal rate per game stands at 1.5, with home games seeing an average of 2.0 goals and away games 0.9 goals.',
   'goal_distribution': 'The goal distribution stats show that the team typically scores the most between minutes 46-60 with a 23.21% rate. They are also the most vulnerable in the same time frame, conceding 27.27% of their total g